# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')


# check get key
if OPENAI_API_KEY:
    print("API key loaded successfully")
else:
    print("API key NOT found")

API key loaded successfully


# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI() 

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [ ]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [ ]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [ ]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

In [ ]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [ ]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [ ]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Few Shots for classification.


In [ ]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

0 shot

In [4]:
context_user = [
    {'role':'system', 'content':'You are chatgpt 3.5.'}
]

In [5]:
print(return_OAIResponse("Remove the vowels from my sentence: mississippi pizza with ananas is great", context_user))

"mspzz wth nn s grt"


1 - shot. Only 1 example. No system instructions

In [6]:
context_user = [
    {'role':'system', 'content':'You are chatgpt 3.5.'},
    {
        'role': 'user',
        'content': 'Remove the vowels from Hello World'
    },
    {
        'role': 'assistant',
        'content': 'hll wrld'
    },
]

In [7]:
print(return_OAIResponse("Remove the vowels from my sentence: mississippi pizza with ananas is great", context_user))

mssssppzz wth nnns s grt


1 - shot. Only instructions. No examples

In [8]:
context_user = [
    {
        'role': 'system',
        'content': """You apply the Vowel Collapse Cipher.

Rules:
- Remove all vowels (a, e, i, o, u).
- Keep consonants in original order.
- Keep spaces.
- Output must be lowercase.
- Do not include punctuation.
- Return ONLY the transformed sentence.
"""
    },
]

In [9]:
print(return_OAIResponse("Remove the vowels from my sentence: mississippi pizza with ananas is great", context_user))

msspp pz wz nnns s grt


2 shot. Instructions and 2 examples

In [10]:
context_user = [
    {
        'role': 'system',
        'content': """You apply the Vowel Collapse Cipher.

Rules:
- Remove all vowels (a, e, i, o, u).
- Keep consonants in original order.
- Keep spaces.
- Output must be lowercase.
- Do not include punctuation.
- Return ONLY the transformed sentence.
"""
    },
    {
        'role': 'user',
        'content': 'Hello World'
    },
    {
        'role': 'assistant',
        'content': 'hll wrld'
    },
    {
        'role': 'user',
        'content': 'I love pizza'
    },
    {
        'role': 'assistant',
        'content': 'lv pzz'
    },
]

In [11]:
print(return_OAIResponse("Remove the vowels from my sentence: mississippi pizza with ananas is great", context_user))

msssspp pz wz nnns s grt


3 shot with exact example

In [12]:
context_user = [
    {
        'role': 'system',
        'content': """You apply the Vowel Collapse Cipher.

Rules:
- Remove all vowels (a, e, i, o, u).
- Keep consonants in original order.
- Keep spaces.
- Output must be lowercase.
- Do not include punctuation.
- Return ONLY the transformed sentence.
"""
    },
    {
        'role': 'user',
        'content': 'Hello World'
    },
    {
        'role': 'assistant',
        'content': 'hll wrld'
    },
    {
        'role': 'user',
        'content': 'I love pizza'
    },
    {
        'role': 'assistant',
        'content': 'lv pzz'
    },
    {
        'role': 'user',
        'content': 'mississippi pizza with ananas is great'
    },
    {
        'role': 'assistant',
        'content': 'msssspp pzz wth nns s grt'
    }
]

In [13]:
print(return_OAIResponse("Remove the vowels from my sentence: mississippi pizza with ananas is great", context_user))

msssspp pzz wth nns s grt


Findings

From the experiments, we observe that GPT-3.5 can handle simple rule-based transformations reasonably well in a zero-shot setting when the instruction is explicit and concrete (e.g., “remove the vowels”). However, even in this relatively simple task, small inconsistencies can appear: spacing issues, minor formatting deviations, or subtle differences in how strictly the model interprets the rule. This shows that while GPT-3.5 has strong pattern-recognition capabilities, its adherence to precise procedural constraints is not always perfectly stable without additional grounding.

When we move to one-shot and two-shot prompting, the model’s behavior becomes more consistent and predictable. Providing examples reduces ambiguity about the expected transformation pattern and output format. With two examples in particular, GPT-3.5 generalizes the rule more reliably and minimizes small structural errors. The key learning is that few-shot prompting does not necessarily teach the model new knowledge, but it significantly improves rule alignment, output consistency, and compliance with formatting constraints. In structured or symbolic tasks, additional examples act as anchors that stabilize the model’s interpretation of the instructions.